In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import requests
from IPython.display import SVG, display
import pandas as pd

In [36]:
def player_history(soup):
    tables = soup.find_all('table')
    
    batting_history = []
    bowling_history = []

    for num, table in enumerate(tables):
        titles = table.find_all('th')
        for title in titles:
            x = title.get_text().strip()
            if num == 0 and x != "":
                batting_history.append(x)
            elif x != "":
                bowling_history.append(x)
#     print(batting_history)
#     print(bowling_history)
    return [batting_history,bowling_history]

In [72]:
def player_formats(soup):
    tables = soup.find_all('table')

    played_batting_format = []
    
    played_bowling_format = []
    
    for num,table in enumerate(tables):
        rows = table.find_all('tr')
        for row in rows:
            first_box = row.find_all('td')
            if first_box:
                x = first_box[0].get_text().strip()
                if num == 0:
                    played_batting_format.append(x)
                else:
                    played_bowling_format.append(x)
#     print(played_batting_format)
#     print(played_bowling_format)
    return [played_batting_format,played_bowling_format]
# player_formats(soup)

In [73]:
def batting_and_bowling_stats(soup):
    
    tables = soup.find_all('table')
    batting_stats = []
    bowling_stats = []

    for num,table in enumerate(tables):
        rows = table.find_all('tr')
        for row in rows:
            arr = []
            first_box = row.find_all('td')
            for box in first_box[1:]:
                x = box.get_text().strip()
                arr.append(x)
            if num == 0 and arr:
                batting_stats.append(arr)
            elif arr:
                bowling_stats.append(arr)
#     print(batting_stats)
#     print(bowling_stats)
    return [batting_stats,bowling_stats]

In [74]:
import pandas as pd

def calculate_performance(batting_df, bowling_df, formats, fixed_max_batting, fixed_max_bowling):
    # Apply function to calculate performance for each format
    batting_df['Batting_Performance'] = batting_df.apply(calculate_batting_performance, axis=1)
    bowling_df['Bowling_Performance'] = bowling_df.apply(calculate_bowling_performance, axis=1)

    # Normalize the batting performances to get percentages out of 100 using the fixed max value
    batting_df['Batting_Percentage'] = (batting_df['Batting_Performance'] / fixed_max_batting) * 100

    # Normalize the bowling performances to get percentages out of 100 using the fixed max value
    bowling_df['Bowling_Percentage'] = (bowling_df['Bowling_Performance'] / fixed_max_bowling) * 100

    # Add the format names to the DataFrames
    batting_df['Format'] = formats
    bowling_df['Format'] = formats

    # Merge batting and bowling data into a single DataFrame to avoid duplicating the format column
    combined_df = pd.merge(batting_df[['Format', 'Batting_Percentage']], 
                           bowling_df[['Format', 'Bowling_Percentage']], 
                           on='Format')

    combined_df.index += 1
    
    # Display results
    print("Performance (out of 100%):")
    print()
    print()
    print(combined_df)
    print()

# Function to calculate performance percentages for batting
def calculate_batting_performance(row):
    avg = row['Avg']
    strike_rate = row['SR']
    centuries = row['100']

    # Weightings for batting
    avg_weight = float(avg) * 0.5
    sr_weight = float(strike_rate) * 0.3
    centuries_weight = int(centuries) * 2

    return avg_weight + sr_weight + centuries_weight

# Function to calculate performance percentages for bowling
def calculate_bowling_performance(row):
    try:
        econ = float(row['Econ']) if row['Econ'] is not None else 0
        wickets = float(row['Wkts']) if row['Wkts'] is not None else 0
    except ValueError:
        econ = 0
        wickets = 0

    # Weightings for bowling
    econ_weight = (3.0 / econ) * 50 if econ != 0 else 0
    wickets_weight = wickets * 2

    return econ_weight + wickets_weight

# Example usage:
formats = ['Test', 'ODI', 'T20I', 'IPL']  # Example format list

batting_df = pd.DataFrame(batting_data)
bowling_df = pd.DataFrame(bowling_data)

# Choose fixed maximum performance values
fixed_max_batting = 200  # Example maximum for batting
fixed_max_bowling = 150  # Example maximum for bowling


In [75]:
def find_stats(soup):
    find_name = soup.find("h1", {"itemprop": "name"})
    print(find_name.text)
    
    batting_and_bowling_history = player_history(soup)
    batting_and_bowling_formats = player_formats(soup)
    batting_and_bowling_statas = batting_and_bowling_stats(soup)
    
    batting_df = pd.DataFrame(batting_and_bowling_statas[0], columns=batting_and_bowling_history[0], index=batting_and_bowling_formats[0])
    bowling_df = pd.DataFrame(batting_and_bowling_statas[1], columns=batting_and_bowling_history[1], index=batting_and_bowling_formats[1])
    print(batting_df)
    print()
    print(bowling_df)
    print()
    
    fixed_max_batting = 200
    fixed_max_bowling = 150 
    calculate_performance(batting_df,bowling_df,batting_and_bowling_formats[0],fixed_max_batting,fixed_max_bowling)
    
    

In [76]:
# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Define the Google search URL
google_search_url = "https://www.google.com/"

# Define the list of names
names = ['MS DHONI', 'VIRAT KOHLI', 'JADEJA','SACHIN']


for name in names:
    # Open Google
    driver.get(google_search_url)
    
    search_box = driver.find_element("name", "q")  # Google's search box name is 'q'
#     print(search_box)
    search_box.send_keys(f"{name} Crickbuzz")
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(2)
    
    # Find all search results
    search_results = driver.find_elements("css selector", "a[jsname='UWckNb']")  # Find all <a jsname> tags
    
    if search_results:
        first_result = search_results[0]  # Get the first search result
        result_url = first_result.get_attribute("href")  # Get the URL of the search result
        
        # Click the search result
        first_result.click()
        
        # Wait for the new page to load
        time.sleep(2)
        
        # Extract page content
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Call your function to find stats
        find_stats(soup)
        print()
        print()
    else:
        print(f"No search results found for {name}")

# Close the driver
driver.quit()


MS Dhoni
        M  Inn  NO   Runs   HS    Avg     BF      SR 100 200  50   4s   6s
Test   90  144  16   4876  224  38.09   8248   59.12   6   1  33  544   78
ODI   350  297  84  10773  183  50.58  12303   87.56  10   0  73  826  229
T20I   98   85  42   1617   56   37.6   1282  126.13   0   0   2  116   52
IPL   264  229  95   5243   84  39.13   3812  137.54   0   0  24  363  252

        M Inn   B Runs Wkts   BBI   BBM  Econ   Avg    SR 5W 10W
Test   90   7  96   67    0   0/1   0/1  4.19   0.0   0.0  0   0
ODI   350   2  36   31    1  1/14  1/14  5.17  31.0  36.0  0   0
T20I   98   -   -    -    -     -     -     -     -     -  -   -
IPL   264   -   -    -    -     -     -     -     -     -  -   -

Performance (out of 100%):


  Format  Batting_Percentage  Bowling_Percentage
1   Test             24.3905           23.866348
2    ODI             35.7790           20.675693
3   T20I             28.3195            0.000000
4    IPL             30.4135            0.000000



Virat Kohli
